In [18]:
# https://pypi.org/project/exif/
# pip install exif

# https://pypi.org/project/pymediainfo/
# pip install pymediainfo

In [64]:
import pandas as pd
# import PIL.Image
from exif import Image
from pprint import pprint
from pymediainfo import MediaInfo
from pathlib import Path
import numpy as np
import os
import re

In [42]:
# path = Path('G:\Google Photo_extr\Takeout\Google Фото')
path = Path(r'D:\Рабочая\!_Scripts\change_file_creation_time')

In [3]:
# %%time
# df = pd.DataFrame(columns=['name', 'extension', 'path'])
# for file_path in path.glob('**/*.*'):
#     extension = str(file_path.name).split('.')[-1]
#     name = file_path.name
#     parent_path = file_path.parent
#     df.loc[len(df)] = [name, extension, parent_path]
# df.head()

Wall time: 43 ms


,name,extension,path
0,.git,git,D:\Рабочая\!_Scripts\change_file_creation_time
1,.ipynb_checkpoints,ipynb_checkpoints,D:\Рабочая\!_Scripts\change_file_creation_time
2,analysis.ipynb,ipynb,D:\Рабочая\!_Scripts\change_file_creation_time
3,changeFileCreationTime.py,py,D:\Рабочая\!_Scripts\change_file_creation_time
4,IMG_20211123_162603.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time


In [43]:
%%time
df = pd.DataFrame(columns=['name', 'extension', 'path'])
for file_path in path.glob('**/*.*'):
    if file_path.is_file():
        extension = file_path.suffix.lower()
        name = file_path.name
        parent_path = file_path.parent
        df.loc[len(df)] = [name, extension, parent_path]
df.head()

Wall time: 32 ms


,name,extension,path
0,analysis.ipynb,.ipynb,D:\Рабочая\!_Scripts\change_file_creation_time
1,changeFileCreationTime.py,.py,D:\Рабочая\!_Scripts\change_file_creation_time
2,IMG_20211123_162603.jpg,.jpg,D:\Рабочая\!_Scripts\change_file_creation_time
3,IMG_20211123_162801.jpg,.jpg,D:\Рабочая\!_Scripts\change_file_creation_time
4,VID_20190106_162804.mp4,.mp4,D:\Рабочая\!_Scripts\change_file_creation_time


In [44]:
# посмотрим сколько каких файлов по типам расширения у нас есть
df.extension.value_counts()

.sample    13
.jpg        2
.ipynb      2
.py         1
.txt        1
.mp4        1
Name: extension, dtype: int64

In [70]:
df.query('extension in @video_ext or extension in @photo_ext')

,name,extension,path
2,IMG_20211123_162603.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time
3,IMG_20211123_162801.jpg,jpg,D:\Рабочая\!_Scripts\change_file_creation_time
6,VID_20190106_162804.mp4,mp4,D:\Рабочая\!_Scripts\change_file_creation_time


In [69]:
# списки расширений для видео и фото
video_ext = np.array(['mp4'])
photo_ext = np.array(['jpg'])

In [46]:
media_info = MediaInfo.parse("VID_20190106_162804.mp4")
for track in media_info.tracks:
    if track.track_type == "Video":
        exif_vdata = track.to_data()
vdata = exif_vdata['tagged_date']
vdata

'UTC 2019-01-07 00:28:08'

In [50]:
pd.to_datetime(vdata[4:])

Timestamp('2019-01-07 00:28:08')

In [55]:
time_zone = pd.to_timedelta(7, unit='h')

In [56]:
pd.to_datetime(vdata[4:]) + time_zone

Timestamp('2019-01-07 07:28:08')

In [59]:
with open('IMG_20211123_162603.jpg', 'rb') as image_file:
    my_image = Image(image_file)
my_image.has_exif

True

In [72]:
# список всех методов
# my_image.list_all()

In [63]:
fdata = my_image.datetime
fdata

'2021:11:23 16:26:05'

In [71]:
fpattern = re.compile('(\d\d\d\d:\d\d:\d\d) (\d\d:\d\d:\d\d)') 
fpattern.findall(fdata)[0][0].replace(':','-') + ' ' + fpattern.findall(fdata)[0][1]

'2021-11-23 16:26:05'

In [25]:
my_image.datetime_original

'2021:11:23 16:26:05'

In [26]:
my_image.datetime_digitized

'2021:11:23 16:26:05'

In [27]:
my_image.gps_datestamp

'2021:11:23'